In [1]:
from pathlib import Path
import pickle

import numpy as np
import numpy.random as nr

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
# Print out packages versions
print(f'pandas version is: {pd.__version__}')
print(f'numpy version is: {np.__version__}')
print(f'matplotlib version is: {matplotlib.__version__}')
print(f'sklearn version is: {sklearn.__version__}')
print(f'xgboost version is: {xgb.__version__}')

pandas version is: 1.1.5
numpy version is: 1.19.5
matplotlib version is: 3.2.2
sklearn version is: 0.22.2.post1
xgboost version is: 0.90


# Helper functions

In [3]:
def replace_nan_inf(df, value=None):
    """
    Replace missing and infinity values.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with values to be replaced.

    value : int, float
        Value to replace any missing or numpy.inf values. Defaults to numpy.nan
    Returns
    -------
    pandas.DataFrame
        Dataframe with missing and infinity values replaced with -999.

    """
    if value is None:
        value = np.nan
    return df.replace(to_replace=[np.nan, np.inf, -np.inf],
                      value=value)


def shift_concat(df, periods=1, fill_value=None):
    """
    Build dataframe of shifted index.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns to be shifted.
    periods : int
        Number of periods to shift. Should be positive.
    fill_value : object, optional
        The scalar value to use for newly introduced missing values. Defaults
        to numpy.nan.

    Returns
    -------
    pandas.DataFrame
        Shifted dataframes concatenated along columns axis.

    Notes
    -------
    Based on Paulo Bestagini's augment_features_window from SEG 2016 ML
    competition.
    https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try01.ipynb

    Example
    -------
    Shift df by one period and concatenate.

    >>> df = pd.DataFrame({'gr': [1.1, 2.1], 'den': [2.1, 2.2]})
    >>> shift_concat(df)
        gr_shifted_1  den_shifted_1  gr   den  gr_shifted_-1   den_shifted_-1
    0      NaN            NaN        1.1  2.1      2.1             2.2
    1      1.1            2.1        2.1  2.2      NaN             NaN

    """
    if fill_value is None:
        fill_value = np.nan

    dfs = []
    for period in range(periods, -1*periods - 1, -1):

        if period == 0:
            dfs.append(df)
            continue

        df_shifted = df.shift(period, fill_value=fill_value)

        df_shifted.columns = [f'{col}_shifted_{str(period)}'
                              for col in df_shifted.columns]

        dfs.append(df_shifted)

    return pd.concat(dfs, axis=1)


def gradient(df, depth_col):
    """
    Calculate the gradient for all features along the provided `depth_col`
    column.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns to be used in the gradient calculation.
    depth_col : str
        Dataframe column name to be used as depth reference.

    Returns
    -------
    pandas.DataFrame
        Gradient of `df` along `depth_col` column. The depth column is not in
        the output dataframe.

    Notes
    -------
    Based on Paulo Bestagini's augment_features_window from SEG 2016 ML
    competition.
    https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try01.ipynb

    Example
    -------
    Calculate gradient of columns along `md`.

    >>> df = pd.DataFrame({'gr': [100.1, 100.2, 100.3],
                          'den': [2.1, 2.2, 2.3],
                          'md': [500, 500.5, 501]})
    >>> gradient(df, 'md')
        gr  den
    0  NaN  NaN
    1  0.2  0.2
    2  0.2  0.2

    """
    depth_diff = df[depth_col].diff()

    denom_zeros = np.isclose(depth_diff, 0)
    depth_diff[denom_zeros] = 0.001

    df_diff = df.drop(depth_col, axis=1)
    df_diff = df_diff.diff()

    # Add suffix to column names
    df_diff.columns = [f'{col}_gradient' for col in df_diff.columns]

    return df_diff.divide(depth_diff, axis=0)


def shift_concat_gradient(df, depth_col, well_col, cat_cols, periods=1, fill_value=None):
    """
    Augment features using `shif_concat` and `gradient`.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns to be augmented.
    depth_col : str
        Dataframe column name to be used as depth reference.
    well_col : str
        Dataframe column name to be used as well reference.
    cat_cols: list of str
        Encoded column names. The gradient calculation is not applied to these
        columns.
    periods : int
        Number of periods to shift. Should be positive.
    fill_value : object, optional
        The scalar value to use for newly introduced missing values. Defaults
        to numpy.nan.

    Returns
    -------
    pandas.DataFrame
        Augmented dataframe.

    Notes
    -------
    Based on Paulo Bestagini's augment_features_window from SEG 2016 ML
    competition.
    https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try01.ipynb

    Example
    -------
    Augment features of `df` by shifting and taking the gradient.

    >>> df = pd.DataFrame({'gr': [100.1, 100.2, 100.3, 20.1, 20.2, 20.3],
                          'den': [2.1, 2.2, 2.3, 1.7, 1.8, 1.9],
                           'md': [500, 500.5, 501, 1000, 1000.05, 1001],
                         'well': [1, 1, 1, 2, 2, 2]})
    >>> shift_concat_gradient(df, 'md', 'well', periods=1, fill_value=None)
        gr_shifted_1  den_shifted_1     gr    den  ...  well   md    gr_gradient  den_gradient
    0         NaN          NaN         100.1  2.1  ...   1   500.00        NaN           NaN
    1       100.1          2.1         100.2  2.2  ...   1   500.50   0.200000      0.200000
    2       100.2          2.2         100.3  2.3  ...   1   501.00   0.200000      0.200000
    3         NaN          NaN          20.1  1.7  ...   2  1000.00        NaN           NaN
    4        20.1          1.7          20.2  1.8  ...   2  1000.05   2.000000      2.000000
    5        20.2          1.8          20.3  1.9  ...   2  1001.00   0.105263      0.105263

    """
    # TODO 'Consider filling missing values created here with DataFrame.fillna'

    # Columns to apply gradient operation
    cat_cols.append(well_col)
    gradient_cols = [col for col in df.columns if col not in cat_cols]

    # Don't shift depth
    depth = df.loc[:, depth_col]

    grouped = df.groupby(well_col, sort=False)

    df_aug_groups = []
    for name, group in grouped:
        shift_cols_df = group.drop([well_col, depth_col], axis=1)

        group_shift = shift_concat(shift_cols_df,
                                   periods=periods,
                                   fill_value=fill_value)

        # Add back the well name and depth
        group_shift[well_col] = name
        group_shift[depth_col] = depth

        group_gradient = group.loc[:, gradient_cols]

        group_gradient = gradient(group_gradient, depth_col)

        group_aug = pd.concat([group_shift, group_gradient], axis=1)

        df_aug_groups.append(group_aug)

    return pd.concat(df_aug_groups)


def score(y_true, y_pred, scoring_matrix):
    """
    Competition scoring function.

    Parameters
    ----------
    y_true : pandas.Series
        Ground truth (correct) target values.
    y_pred : pandas.Series
        Estimated targets as returned by a classifier.
    scoring_matrix : numpy.array
        Competition scoring matrix.

    Returns
    ----------
    float
        2020 FORCE ML lithology competition custome score.

    """
    S = 0.0

    for true_val, pred_val in zip(y_true, y_pred):
        S -= scoring_matrix[true_val, pred_val]

    return S/y_true.shape[0]


def show_evaluation(y_true, y_pred):
    """
    Print model performance and evaluation.

    Parameters
    ----------
    y_true : pandas.Series
        Ground truth (correct) target values.
    y_pred: pandas.Series
        Estimated targets as returned by a classifier.

    """
    print(f'Competition score: {score(y_true, y_pred)}')
    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
    print(f'F1: {f1_score(y_true, y_pred, average="weighted")}')


def build_encoding_map(series):
    """
    Build dictionary with the mapping of series unique values to encoded
    values.

    Parameters
    ----------
    series : pandas.Series
        Series with categories to be encoded.

    Returns
    -------
    mapping : dict
        Dictionary mapping unique categories in series to encoded values.

    See Also
    --------
    label_encode_columns : Label encode a dataframe categorical columns.

    """
    unique_values = series.unique()

    mapping = {original: encoded
               for encoded, original in enumerate(unique_values)
               if original is not np.nan}

    return mapping


def label_encode_columns(df, cat_cols, mappings):
    """
    Label encode a dataframe categorical columns.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns to be encoded.
    cat_cols: list of str
        Column names to be encoded.
    mappings: dict of dict
        Dictionary containing a key-value mapping for each column to be
        encoded.

    Returns
    -------
    df : pandas.DataFrame
        Dataframe with the encoded columns added and the `cat_cols` removed.
    encoded_col_names: list of str
        Encoded column names.

    See Also
    --------
    build_encoding_map : Build a series encoding mapping.

    """
    df = df.copy()

    encoded_col_names = []
    for col in cat_cols:
        new_col = f'{col}_encoded'
        encoded_col_names.append(new_col)

        df[new_col] = df[col].map(mappings[col])

        df.drop(col, axis=1, inplace=True)

    return df, encoded_col_names

# Target maps

In [4]:
KEYS_TO_ORDINAL = {
    30000: 0,
    65030: 1,
    65000: 2,
    80000: 3,
    74000: 4,
    70000: 5,
    70032: 6,
    88000: 7,
    86000: 8,
    99000: 9,
    90000: 10,
    93000: 11
    }


KEYS_TO_LITHOLOGY = {30000: 'Sandstone',
                     65030: 'Sandstone/Shale',
                     65000: 'Shale',
                     80000: 'Marl',
                     74000: 'Dolomite',
                     70000: 'Limestone',
                     70032: 'Chalk',
                     88000: 'Halite',
                     86000: 'Anhydrite',
                     99000: 'Tuff',
                     90000: 'Coal',
                     93000: 'Basement'}

ORDINAL_TO_KEYS = {value: key for key, value in  KEYS_TO_ORDINAL.items()}

ORDINAL_TO_LITHOLOGY = {}
for ordinal_key, key in ORDINAL_TO_KEYS.items():
    ORDINAL_TO_LITHOLOGY[ordinal_key] = KEYS_TO_LITHOLOGY[key]

# Import data

First add a shortcut from the [google drive competition data location](https://drive.google.com/drive/folders/1GIkjq4fwgwbiqVQxYwoJnOJWVobZ91pL) to your own google drive. We will mount this drive, and access the data from it.

We will save the results to a diffent folder, where we have write access.

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
#should be edited to the present working directory of the user
data_source = '/content/drive/My Drive/FORCE 2020 lithofacies prediction from well logs competition/'

In [7]:
penalty_matrix = np.load(data_source + 'penalty_matrix.npy')

train = pd.read_csv(data_source + 'CSV_train.csv', sep=';')

test = pd.read_csv(data_source + 'CSV_test.csv', sep=';')

In [8]:
# Destination folder
out_data_dir = Path('/content/drive/My Drive/lith_pred/')

# Train model

In [9]:
class Model():
    '''
    class to lithology prediction
    '''
    def preprocess(self, df, cat_columns, mappings):
        # Grab model features
        keep_cols = ['WELL', 'DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'GROUP',
                     'FORMATION', 'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR',
                     'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'DRHO']

        df = df.loc[:, keep_cols]

        # Label encode
        df, encoded_col_names = label_encode_columns(df, cat_columns, mappings)
        
        # Augment using Bestagini's functions
        df_preprocesed = shift_concat_gradient(df,
                                               'DEPTH_MD',
                                               'WELL',
                                               encoded_col_names,
                                               periods=1,
                                               fill_value=None)
        
        # Fill NaN's with -999
        df_preprocesed.replace([np.inf, -np.inf], np.nan, inplace=True)
        df_preprocesed.fillna(-999, inplace=True)
       
        return df_preprocesed

    def fit(self, X, y):
        split = 10
        skf = StratifiedKFold(n_splits=split, shuffle=True)

        model = XGBClassifier(n_estimators=100, max_depth=10, booster='gbtree',
                              objective='multi:softprob', learning_rate=0.1, random_state=0,
                              subsample=0.9, colsample_bytree=0.9, tree_method='gpu_hist',
                              eval_metric='mlogloss', verbose=2020, reg_lambda=1500)
        
        models = []
        for fold_number, indices in enumerate(skf.split(X, y)):
            print(f'Fitting fold: {fold_number}')
            
            train_index, test_index = indices

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            model.fit(X_train,
                      y_train,
                      early_stopping_rounds=100,
                      eval_set=[(X_test, y_test)],
                      verbose=100)

            models.append(model)

        return models

    def fit_predict(self, X_train, y_train, test_wells, save_filename):
        # Fit
        models = self.fit(X_train, y_train)

        # Get lithologies probabilities for each model
        models_proba = []
        for model_num, model in enumerate(models):
            model_proba = model.predict_proba(X_test)
            model_classes = [ORDINAL_TO_LITHOLOGY[lith] for lith in model.classes_]

            model_proba_df = pd.DataFrame(model_proba, columns=model_classes)

            model_proba_df['MODEL'] = model_num

            model_proba_df['WELL'] = test_wells
            model_proba_df['DEPTH_MD'] = X_test['DEPTH_MD']

            models_proba.append(model_proba_df)

        models_proba = pd.concat(models_proba, ignore_index=True)

        # Create save directory if it doesn't exists
        if not save_filename.parent.is_dir():
            save_filename.parent.mkdir(parents=True)

        # Save models_proba to CSV
        models_proba.to_csv(save_filename, index=False)

        return models, models_proba

# Prepare train data

In [10]:
y = train['FORCE_2020_LITHOFACIES_LITHOLOGY']
X = train.drop('FORCE_2020_LITHOFACIES_LITHOLOGY', axis=1)

In [11]:
# Map lithology codes to scoring matrix index
y_train = y.map(KEYS_TO_ORDINAL)

In [12]:
model = Model()

In [13]:
X.columns

Index(['WELL', 'DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'GROUP', 'FORMATION',
       'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'SGR', 'NPHI', 'PEF',
       'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC',
       'ROPA', 'RXO', 'FORCE_2020_LITHOFACIES_CONFIDENCE'],
      dtype='object')

In [14]:
cat_columns = ['GROUP', 'FORMATION']

train_mappings = {col: build_encoding_map(X[col]) for col in cat_columns}

X_train = model.preprocess(X, cat_columns, train_mappings)

In [15]:
X_train.drop('WELL', axis=1, inplace=True)

In [16]:
X_train.columns

Index(['X_LOC_shifted_1', 'Y_LOC_shifted_1', 'Z_LOC_shifted_1',
       'CALI_shifted_1', 'RSHA_shifted_1', 'RMED_shifted_1', 'RDEP_shifted_1',
       'RHOB_shifted_1', 'GR_shifted_1', 'NPHI_shifted_1', 'PEF_shifted_1',
       'DTC_shifted_1', 'SP_shifted_1', 'BS_shifted_1', 'DRHO_shifted_1',
       'GROUP_encoded_shifted_1', 'FORMATION_encoded_shifted_1', 'X_LOC',
       'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI',
       'PEF', 'DTC', 'SP', 'BS', 'DRHO', 'GROUP_encoded', 'FORMATION_encoded',
       'X_LOC_shifted_-1', 'Y_LOC_shifted_-1', 'Z_LOC_shifted_-1',
       'CALI_shifted_-1', 'RSHA_shifted_-1', 'RMED_shifted_-1',
       'RDEP_shifted_-1', 'RHOB_shifted_-1', 'GR_shifted_-1',
       'NPHI_shifted_-1', 'PEF_shifted_-1', 'DTC_shifted_-1', 'SP_shifted_-1',
       'BS_shifted_-1', 'DRHO_shifted_-1', 'GROUP_encoded_shifted_-1',
       'FORMATION_encoded_shifted_-1', 'DEPTH_MD', 'X_LOC_gradient',
       'Y_LOC_gradient', 'Z_LOC_gradient', 'CALI_gradient', 'RS

# Prepare test data

In [17]:
test.shape

(136786, 27)

In [18]:
X_test = model.preprocess(test, cat_columns, train_mappings)
X_test.drop('WELL', axis=1, inplace=True)

In [19]:
X_test.shape

(136786, 67)

In [20]:
X_test.columns

Index(['X_LOC_shifted_1', 'Y_LOC_shifted_1', 'Z_LOC_shifted_1',
       'CALI_shifted_1', 'RSHA_shifted_1', 'RMED_shifted_1', 'RDEP_shifted_1',
       'RHOB_shifted_1', 'GR_shifted_1', 'NPHI_shifted_1', 'PEF_shifted_1',
       'DTC_shifted_1', 'SP_shifted_1', 'BS_shifted_1', 'DRHO_shifted_1',
       'GROUP_encoded_shifted_1', 'FORMATION_encoded_shifted_1', 'X_LOC',
       'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI',
       'PEF', 'DTC', 'SP', 'BS', 'DRHO', 'GROUP_encoded', 'FORMATION_encoded',
       'X_LOC_shifted_-1', 'Y_LOC_shifted_-1', 'Z_LOC_shifted_-1',
       'CALI_shifted_-1', 'RSHA_shifted_-1', 'RMED_shifted_-1',
       'RDEP_shifted_-1', 'RHOB_shifted_-1', 'GR_shifted_-1',
       'NPHI_shifted_-1', 'PEF_shifted_-1', 'DTC_shifted_-1', 'SP_shifted_-1',
       'BS_shifted_-1', 'DRHO_shifted_-1', 'GROUP_encoded_shifted_-1',
       'FORMATION_encoded_shifted_-1', 'DEPTH_MD', 'X_LOC_gradient',
       'Y_LOC_gradient', 'Z_LOC_gradient', 'CALI_gradient', 'RS

# Fit and predict

In [21]:
save_filename = out_data_dir / 'model_proba/models_proba_without_nans.csv'

In [22]:
test_wells = test['WELL']

In [23]:
if save_filename.is_file():
    models_proba = pd.read_csv(save_filename)

else:
    models, models_proba = model.fit_predict(X_train, y_train, test_wells, save_filename)

Fitting fold: 0
[0]	validation_0-mlogloss:2.16303
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[99]	validation_0-mlogloss:0.321278
Fitting fold: 1
[0]	validation_0-mlogloss:2.16295
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[99]	validation_0-mlogloss:0.320756
Fitting fold: 2
[0]	validation_0-mlogloss:2.16313
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[99]	validation_0-mlogloss:0.322432
Fitting fold: 3
[0]	validation_0-mlogloss:2.16257
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[99]	validation_0-mlogloss:0.320238
Fitting fold: 4
[0]	validation_0-mlogloss:2.16306
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[99]	validation_0-mlogloss:0.318843
Fitting fold: 5
[0]	validation_0-mlogloss:2.16352
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[99]	validation_0-mlogloss:0.325207
Fitting fold: 6
[0]	validation_0-mlogloss:2.1625
Will train unti

# Explore models proba

In [24]:
models_proba.sample(10)

,Sandstone,Sandstone/Shale,Shale,Marl,Dolomite,Limestone,Chalk,Halite,Anhydrite,Tuff,Coal,Basement,MODEL,WELL,DEPTH_MD
53873,0.019424,0.052619,0.915707,0.001129,0.000759,0.006059,0.000435,0.000384,0.000388,0.002122,0.000612,0.000364,0,29/3-1,2027.442001
1335684,0.024064,0.222123,0.700815,0.012785,0.002585,0.022039,0.001718,0.001517,0.001531,0.002100,0.007284,0.001437,9,34/3-3 A,3790.353976
812575,0.658171,0.133824,0.069915,0.048588,0.004533,0.065461,0.003033,0.002678,0.002703,0.003489,0.005068,0.002537,5,35/6-2 S,3105.912467
981590,0.036259,0.031710,0.652066,0.009469,0.017371,0.236397,0.002550,0.001557,0.001572,0.004571,0.005003,0.001476,7,25/10-10,1669.030400
728093,0.909056,0.065382,0.016149,0.001795,0.000789,0.003806,0.000450,0.000397,0.000401,0.000517,0.000882,0.000376,5,25/5-3,2729.459190
1091026,0.037830,0.130489,0.561865,0.123477,0.002978,0.131164,0.002087,0.001842,0.001860,0.002400,0.002264,0.001745,7,35/9-8,2690.109600
341685,0.018987,0.347525,0.614824,0.003241,0.001867,0.006020,0.001124,0.000992,0.001001,0.001292,0.002186,0.000940,2,29/3-1,4203.474001
370087,0.010075,0.073573,0.888989,0.007215,0.001471,0.012962,0.000978,0.000863,0.000871,0.001125,0.001061,0.000818,2,34/3-3 A,2559.457975
829826,0.009095,0.017128,0.955771,0.001508,0.001038,0.006612,0.001749,0.000528,0.000551,0.004861,0.000658,0.000500,6,15/9-14,1866.716001
1214658,0.024954,0.029989,0.867703,0.050021,0.002257,0.016308,0.001500,0.001324,0.001337,0.001725,0.001628,0.001255,8,35/6-2 S,1846.136467


In [25]:
models_proba.describe()

,Sandstone,Sandstone/Shale,Shale,Marl,Dolomite,Limestone,Chalk,Halite,Anhydrite,Tuff,Coal,Basement,MODEL,DEPTH_MD
count,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06,1.367860e+06
mean,1.751052e-01,1.266453e-01,5.908246e-01,2.755567e-02,2.701611e-03,5.250223e-02,5.209651e-03,1.308346e-03,1.376494e-03,9.369767e-03,5.544272e-03,1.158950e-03,4.500000e+00,2.501137e+03
std,2.996933e-01,1.617590e-01,3.555661e-01,7.794026e-02,2.857741e-03,1.217339e-01,3.160417e-02,2.304006e-03,3.845067e-03,6.896925e-02,3.828089e-02,1.487847e-03,2.872282e+00,1.043242e+03
min,7.387240e-04,1.779967e-03,1.360025e-03,2.691919e-04,2.565342e-04,6.648308e-04,2.182762e-04,1.359335e-04,1.418189e-04,2.261239e-04,1.732718e-04,1.288208e-04,0.000000e+00,2.272960e+02
25%,1.080134e-02,2.343058e-02,2.013550e-01,1.648624e-03,1.047142e-03,4.598122e-03,7.654973e-04,5.462472e-04,5.537661e-04,9.850349e-04,8.352420e-04,5.097591e-04,2.000000e+00,1.707942e+03
50%,2.879412e-02,6.571491e-02,7.375791e-01,4.709121e-03,1.979968e-03,1.246933e-02,1.360365e-03,9.781686e-04,9.939791e-04,1.797776e-03,1.641484e-03,9.067717e-04,4.500000e+00,2.471824e+03
75%,1.212957e-01,1.529626e-01,9.032147e-01,1.736446e-02,3.278582e-03,3.647524e-02,2.187126e-03,1.607892e-03,1.657933e-03,2.896690e-03,3.501062e-03,1.506258e-03,7.000000e+00,3.294656e+03
max,9.917731e-01,9.489457e-01,9.936102e-01,8.237378e-01,7.459820e-02,9.767642e-01,5.097609e-01,9.177267e-02,3.372239e-01,9.523786e-01,9.399152e-01,6.020198e-02,9.000000e+00,5.007418e+03
